In [ ]:
import json
import os
from pathlib import Path
import shutil
import yaml
from ultralytics import RTDETR

# coco 형식 데이터 -> yolo 형식으로 변환
def convert_coco_to_yolo(coco_file, output_dir, dataset_type, base_dir):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)
    
    images_dir = os.path.join(output_dir, dataset_type, 'images')
    labels_dir = os.path.join(output_dir, dataset_type, 'labels')
    os.makedirs(images_dir, exist_ok=True) 
    os.makedirs(labels_dir, exist_ok=True) 
    
    image_annotations = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id'] 
        if image_id not in image_annotations:
            image_annotations[image_id] = [] 
        image_annotations[image_id].append(ann)
    
    # image 정보 순회하며 yolo 형식으로 변환
    for img in coco_data['images']:
        img_id = img['id'] # image id
        img_name = img['file_name'] # image file_name
        img_width = img['width'] # image width
        img_height = img['height'] # image height
        
        src_image = os.path.join(base_dir, img_name)
        dst_image = os.path.join(images_dir, os.path.basename(img_name))
        shutil.copy(src_image, dst_image)
        
        # YOLO 형식 라벨 파일 생성
        label_file = os.path.join(labels_dir, Path(img_name).stem + '.txt')
        with open(label_file, 'w') as f:
            if img_id in image_annotations:
                for ann in image_annotations[img_id]:
                    category_id = ann['category_id']
                    bbox = ann['bbox']
                    # YOLO 형식의 (x_center, y_center, width, height)로 변환
                    x_center = (bbox[0] + bbox[2] / 2) / img_width
                    y_center = (bbox[1] + bbox[3] / 2) / img_height
                    width = bbox[2] / img_width
                    height = bbox[3] / img_height
                    f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")
    
    print(f"Conversion completed for {dataset_type} dataset")

# YOLO datasets에 맞는 YAML file 생성
def create_dataset_yaml(output_dir, train_dir, val_dir, class_names):
    yaml_content = {
        'path': output_dir, # 데이터셋 생성 dir
        'train': train_dir, # train_dir
        'val': val_dir, # val_dir
        'nc': len(class_names), # class 개수
        'names': class_names # 클래스 이름 목록
    }
    
    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)
    
    return yaml_path

def main():
    base_dir = '/data/ephemeral/home/dataset' # 기본 데이터셋 경로
    folds_dir = os.path.join(base_dir, 'folds') # cv fold 데이터 경로
    yolo_dataset_dir = '/data/ephemeral/home/dataset/RTDETR_dataset'
    
    class_names = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
    
    # 5 fold cv 수행
    for fold in range(5):
        fold_dir = os.path.join(yolo_dataset_dir, f'fold_{fold}')
        os.makedirs(fold_dir, exist_ok=True)
        
        # 학습 데이터 coco -> yolo 반환
        train_json = os.path.join(folds_dir, f'train_fold{fold}.json')
        convert_coco_to_yolo(train_json, fold_dir, 'train', base_dir)
        
        # 검증 데이터 coco -> yolo 반환
        val_json = os.path.join(folds_dir, f'val_fold{fold}.json')
        convert_coco_to_yolo(val_json, fold_dir, 'val', base_dir)
        
        # 폴드 데이터에 맞는 yaml 파일 생성
        train_dir = os.path.join(fold_dir, 'train', 'images')
        val_dir = os.path.join(fold_dir, 'val', 'images')
        yaml_path = create_dataset_yaml(fold_dir, train_dir, val_dir, class_names)
        
        print(f"RTDETR format dataset for fold {fold} has been created at: {fold_dir}")
        
        # YOLO 모델 학습
        model = RTDETR('rtdetr-x.pt')
        model.train(
            data=yaml_path, 
            epochs=30, 
            project="RTDETR", 
            imgsz=640,  # 입력 이미지 크기를 1024x1024로 설정
            batch=16,
            optimizer='AdamW',  # 옵티마이저를 AdamW로 설정
            lr0=1e-3,
            name=f"RTDETR_epoch100_fold{fold}",
            patience=20, # early stopping 20
            iou=0.5,
            label_smoothing=0.1
            )
        print(f"Training completed for fold {fold}")

if __name__ == "__main__":
    main()

Conversion completed for train dataset
Conversion completed for val dataset
RTDETR format dataset for fold 0 has been created at: /data/ephemeral/home/dataset/RTDETR_dataset/fold_0
Ultralytics 8.3.15 🚀 Python-3.10.13 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=rtdetr-x.pt, data=/data/ephemeral/home/dataset/RTDETR_dataset/fold_0/data.yaml, epochs=30, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=RTDETR, name=RTDETR_epoch100_fold04, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: leejh_00 (iron8233-naver). Use `wandb login --relogin` to force relogin


AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed ✅


train: Scanning /data/ephemeral/home/dataset/RTDETR_dataset/fold_0/train/labels.cache... 3901 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3901/3901 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/ephemeral/home/dataset/RTDETR_dataset/fold_0/train/images/4041.jpg: 1 duplicate labels removed


...

  0%|          | 0/243 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
      18/30      19.6G     0.3762     0.9563     0.2019         51        640: 100%|██████████| 243/243 [02:19<00:00,  1.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:10<00:00,  3.11it/s]


                   all       1005       4746       0.56      0.411      0.423      0.348

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/243 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
      19/30      19.7G     0.3648     0.9316     0.1955         72        640: 100%|██████████| 243/243 [02:19<00:00,  1.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:10<00:00,  3.12it/s]


                   all       1005       4746      0.612      0.438      0.453      0.376

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/243 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
      20/30      19.5G     0.3839     0.8871     0.2075        120        640:  21%|██▏       | 52/243 [00:29<01:46,  1.80it/s]